In [1]:
import os
import gc
import time
import numpy as np
import pandas as pd
from contextlib import contextmanager
import multiprocessing as mp
from functools import partial
from scipy.stats import kurtosis, iqr, skew
from lightgbm import LGBMClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
data = pd.read_csv('../input/clean_data.csv', nrows= None)

# How to normalize

In [16]:
"""
df = data[data['TARGET'].notnull()]
y = df["TARGET"].copy()
test_df = data[data['TARGET'].isnull()]
print("Train/valid shape: {}, test shape: {}".format(df.shape, test_df.shape))

del_features = ['TARGET', 'SK_ID_CURR', 'SK_ID_BUREAU', 'SK_ID_PREV', 'index', 'level_0']
predictors = list(filter(lambda v: v not in del_features, df.columns))


train=df[predictors]
test=test_df[predictors]


train = train.fillna(train.mean(), inplace=True)
test= test.fillna(test.mean(), inplace=True)

train=train.values
test=test.values

train = (train - train.min())/(train.max() - train.min())
test = (test - test.min())/(test.max() - test.min())
"""

'\ndf = data[data[\'TARGET\'].notnull()]\ny = df["TARGET"].copy()\ntest_df = data[data[\'TARGET\'].isnull()]\nprint("Train/valid shape: {}, test shape: {}".format(df.shape, test_df.shape))\n\ndel_features = [\'TARGET\', \'SK_ID_CURR\', \'SK_ID_BUREAU\', \'SK_ID_PREV\', \'index\', \'level_0\']\npredictors = list(filter(lambda v: v not in del_features, df.columns))\n\n\ntrain=df[predictors]\ntest=test_df[predictors]\n\n\ntrain = train.fillna(train.mean(), inplace=True)\ntest= test.fillna(test.mean(), inplace=True)\n\ntrain=train.values\ntest=test.values\n\ntrain = (train - train.min())/(train.max() - train.min())\ntest = (test - test.min())/(test.max() - test.min())\n'

In [3]:
df = data[data['TARGET'].notnull()]
y = df["TARGET"].copy()
test_df = data[data['TARGET'].isnull()]

In [4]:
print("Train/valid shape: {}, test shape: {}".format(df.shape, test_df.shape))

del_features = ['TARGET', 'SK_ID_CURR', 'SK_ID_BUREAU', 'SK_ID_PREV', 'index', 'level_0']
predictors = list(filter(lambda v: v not in del_features, df.columns))


Train/valid shape: (307506, 659), test shape: (48744, 659)


In [10]:
train=df[predictors]
test=test_df[predictors]

In [11]:
# Fill Missing data
train = train.fillna(train.mean())
test= test.fillna(test.mean())

In [12]:
train

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,...,CURRENT_TO_APPROVED_ANNUITY_MEAN_RATIO,PAYMENT_MIN_TO_ANNUITY_RATIO,PAYMENT_MAX_TO_ANNUITY_RATIO,PAYMENT_MEAN_TO_ANNUITY_RATIO,CTA_CREDIT_TO_ANNUITY_MAX_RATIO,CTA_CREDIT_TO_ANNUITY_MEAN_RATIO,DAYS_DECISION_MEAN_TO_BIRTH,DAYS_CREDIT_MEAN_TO_BIRTH,DAYS_DECISION_MEAN_TO_EMPLOYED,DAYS_CREDIT_MEAN_TO_EMPLOYED
0,24700.5,406597.5,351000.0,202500.000,0.000000,0.000000,0.000000,0.024700,0.025000,0.02520,...,0.374500,0.374500,2.150000,0.468000,1.175716,1.175716,0.064000,0.092400,0.951334,1.372057
1,35698.5,1293502.5,1129500.0,270000.000,0.000000,0.000000,0.000000,0.095900,0.096800,0.09240,...,1.584000,0.186600,15.710000,1.813000,0.290662,0.239484,0.077800,0.083560,1.098485,1.179082
2,6750.0,135000.0,135000.0,67500.000,0.000000,0.000000,0.000000,0.117445,0.117854,0.11423,...,0.793500,0.793500,1.566000,1.052000,0.187652,0.187652,0.042800,0.045530,3.622222,3.853333
3,29686.5,312682.5,297000.0,135000.000,0.006403,0.265475,0.034363,0.117445,0.117854,0.11423,...,0.736000,0.083600,23.300000,2.121000,2.643133,1.497703,0.018190,0.069982,0.113722,-inf
4,21865.5,513000.0,513000.0,121500.000,0.000000,0.000000,0.000000,0.117445,0.117854,0.11423,...,0.561500,0.000008,1.037000,0.558600,0.931669,0.538926,0.061340,0.057650,0.402513,0.378209
5,27517.5,490495.5,454500.0,99000.000,0.000000,1.000000,0.000000,0.117445,0.117854,0.11423,...,0.575700,0.008255,15.710000,0.994000,1.112682,0.601797,0.086730,0.044700,0.925378,0.476910
6,41301.0,1560726.0,1395000.0,171000.000,0.000000,1.000000,0.000000,0.117445,0.117854,0.11423,...,0.243400,0.149000,0.420000,0.231700,0.288957,0.186418,0.052220,0.092300,0.229804,0.406230
7,42075.0,1530000.0,1530000.0,360000.000,0.000000,0.000000,0.000000,0.117445,0.117854,0.11423,...,0.653000,0.649400,0.653000,0.652300,0.261158,0.261158,0.056760,0.102900,2.383073,4.319599
8,33826.5,1019610.0,913500.0,112500.000,0.000000,0.000000,0.000000,0.117445,0.117854,0.11423,...,0.541000,0.013290,1.374000,0.335000,0.776959,0.581895,0.099100,0.088200,-inf,-inf
9,20250.0,405000.0,405000.0,135000.000,0.006403,0.265475,0.034363,0.117445,0.117854,0.11423,...,0.390000,0.000289,2.898000,0.516000,0.708382,0.568101,0.069400,0.069982,0.497276,-inf


In [13]:
train = (train - train.min())/(train.max() - train.min())

MemoryError: 

In [9]:
train

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,...,CURRENT_TO_APPROVED_ANNUITY_MEAN_RATIO,PAYMENT_MIN_TO_ANNUITY_RATIO,PAYMENT_MAX_TO_ANNUITY_RATIO,PAYMENT_MEAN_TO_ANNUITY_RATIO,CTA_CREDIT_TO_ANNUITY_MAX_RATIO,CTA_CREDIT_TO_ANNUITY_MEAN_RATIO,DAYS_DECISION_MEAN_TO_BIRTH,DAYS_CREDIT_MEAN_TO_BIRTH,DAYS_DECISION_MEAN_TO_EMPLOYED,DAYS_CREDIT_MEAN_TO_EMPLOYED
0,0.090032,0.090287,0.077441,0.009839,0.000000,0.000000,0.000000,0.024700,0.025000,0.02520,...,0.032118,2.163489e-03,0.006117,0.002704,0.0,0.0,0.219274,0.312796,NaN,NaN
1,0.132924,0.311736,0.271605,0.013594,0.000000,0.000000,0.000000,0.095900,0.096800,0.09240,...,0.135849,1.077990e-03,0.044694,0.010474,0.0,0.0,0.266632,0.282871,NaN,NaN
2,0.020025,0.022472,0.023569,0.002328,0.000000,0.000000,0.000000,0.117445,0.117854,0.11423,...,0.068053,4.584055e-03,0.004455,0.006077,0.0,0.0,0.146521,0.154130,NaN,NaN
3,0.109477,0.066837,0.063973,0.006084,0.001601,0.001017,0.004295,0.117445,0.117854,0.11423,...,0.063122,4.829578e-04,0.066287,0.012253,0.0,0.0,0.062065,0.236905,NaN,NaN
4,0.078975,0.116854,0.117845,0.005333,0.000000,0.000000,0.000000,0.117445,0.117854,0.11423,...,0.048156,4.737146e-08,0.002950,0.003227,0.0,0.0,0.210145,0.195159,NaN,NaN
5,0.101018,0.111235,0.103255,0.004081,0.000000,0.003831,0.000000,0.117445,0.117854,0.11423,...,0.049374,4.768920e-05,0.044694,0.005742,0.0,0.0,0.297278,0.151320,NaN,NaN
6,0.154774,0.378458,0.337823,0.008086,0.000000,0.003831,0.000000,0.117445,0.117854,0.11423,...,0.020875,8.607741e-04,0.001195,0.001338,0.0,0.0,0.178848,0.312458,NaN,NaN
7,0.157792,0.370787,0.371493,0.018601,0.000000,0.000000,0.000000,0.117445,0.117854,0.11423,...,0.056003,3.751589e-03,0.001858,0.003768,0.0,0.0,0.194428,0.348341,NaN,NaN
8,0.125623,0.243348,0.217733,0.004832,0.000000,0.000000,0.000000,0.117445,0.117854,0.11423,...,0.046398,7.677643e-05,0.003909,0.001935,0.0,0.0,0.339729,0.298578,NaN,NaN
9,0.072675,0.089888,0.090909,0.006084,0.001601,0.001017,0.004295,0.117445,0.117854,0.11423,...,0.033448,1.669555e-06,0.008245,0.002981,0.0,0.0,0.237805,0.236905,NaN,NaN


In [3]:
# remove 'TARGET' from the list
del_features_2 = [ 'SK_ID_CURR', 'SK_ID_BUREAU', 'SK_ID_PREV', 'index', 'level_0']
predictors = list(filter(lambda v: v not in del_features_2, data.columns))

In [4]:
data2 = data[predictors]

In [5]:
y = data2["TARGET"].copy()

In [7]:
data3=data2.drop('TARGET',axis=1)

In [8]:
data3.shape

(356250, 657)

In [9]:
data3.head(5)

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,...,CURRENT_TO_APPROVED_ANNUITY_MEAN_RATIO,PAYMENT_MIN_TO_ANNUITY_RATIO,PAYMENT_MAX_TO_ANNUITY_RATIO,PAYMENT_MEAN_TO_ANNUITY_RATIO,CTA_CREDIT_TO_ANNUITY_MAX_RATIO,CTA_CREDIT_TO_ANNUITY_MEAN_RATIO,DAYS_DECISION_MEAN_TO_BIRTH,DAYS_CREDIT_MEAN_TO_BIRTH,DAYS_DECISION_MEAN_TO_EMPLOYED,DAYS_CREDIT_MEAN_TO_EMPLOYED
0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0247,0.0250,0.0252,...,0.3745,0.374500,2.150,0.4680,1.175716,1.175716,0.06400,0.09240,0.951334,1.372057
1,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0959,0.0968,0.0924,...,1.5840,0.186600,15.710,1.8130,0.290662,0.239484,0.07780,0.08356,1.098485,1.179082
2,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,NaN,NaN,NaN,...,0.7935,0.793500,1.566,1.0520,0.187652,0.187652,0.04280,0.04553,3.622222,3.853333
3,29686.5,312682.5,297000.0,135000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.7360,0.083600,23.300,2.1210,2.643133,1.497703,0.01819,NaN,0.113722,NaN
4,21865.5,513000.0,513000.0,121500.0,0.0,0.0,0.0,NaN,NaN,NaN,...,0.5615,0.000008,1.037,0.5586,0.931669,0.538926,0.06134,0.05765,0.402513,0.378209


In [10]:
data3=data3.fillna(data3.mean())

In [13]:
data3.head(20)

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,...,CURRENT_TO_APPROVED_ANNUITY_MEAN_RATIO,PAYMENT_MIN_TO_ANNUITY_RATIO,PAYMENT_MAX_TO_ANNUITY_RATIO,PAYMENT_MEAN_TO_ANNUITY_RATIO,CTA_CREDIT_TO_ANNUITY_MAX_RATIO,CTA_CREDIT_TO_ANNUITY_MEAN_RATIO,DAYS_DECISION_MEAN_TO_BIRTH,DAYS_CREDIT_MEAN_TO_BIRTH,DAYS_DECISION_MEAN_TO_EMPLOYED,DAYS_CREDIT_MEAN_TO_EMPLOYED
0,24700.5,406597.5,351000.0,202500.000,0.000000,0.000000,0.000000,0.024700,0.025000,0.025200,...,0.3745,0.374500,2.1500,0.4680,1.175716,1.175716,0.06400,0.092400,0.951334,1.372057
1,35698.5,1293502.5,1129500.0,270000.000,0.000000,0.000000,0.000000,0.095900,0.096800,0.092400,...,1.5840,0.186600,15.7100,1.8130,0.290662,0.239484,0.07780,0.083560,1.098485,1.179082
2,6750.0,135000.0,135000.0,67500.000,0.000000,0.000000,0.000000,0.118142,0.118553,0.114913,...,0.7935,0.793500,1.5660,1.0520,0.187652,0.187652,0.04280,0.045530,3.622222,3.853333
3,29686.5,312682.5,297000.0,135000.000,0.005809,0.304401,0.029995,0.118142,0.118553,0.114913,...,0.7360,0.083600,23.3000,2.1210,2.643133,1.497703,0.01819,0.070019,0.113722,-inf
4,21865.5,513000.0,513000.0,121500.000,0.000000,0.000000,0.000000,0.118142,0.118553,0.114913,...,0.5615,0.000008,1.0370,0.5586,0.931669,0.538926,0.06134,0.057650,0.402513,0.378209
5,27517.5,490495.5,454500.0,99000.000,0.000000,1.000000,0.000000,0.118142,0.118553,0.114913,...,0.5757,0.008255,15.7100,0.9940,1.112682,0.601797,0.08673,0.044700,0.925378,0.476910
6,41301.0,1560726.0,1395000.0,171000.000,0.000000,1.000000,0.000000,0.118142,0.118553,0.114913,...,0.2434,0.149000,0.4200,0.2317,0.288957,0.186418,0.05222,0.092300,0.229804,0.406230
7,42075.0,1530000.0,1530000.0,360000.000,0.000000,0.000000,0.000000,0.118142,0.118553,0.114913,...,0.6530,0.649400,0.6530,0.6523,0.261158,0.261158,0.05676,0.102900,2.383073,4.319599
8,33826.5,1019610.0,913500.0,112500.000,0.000000,0.000000,0.000000,0.118142,0.118553,0.114913,...,0.5410,0.013290,1.3740,0.3350,0.776959,0.581895,0.09910,0.088200,-inf,-inf
9,20250.0,405000.0,405000.0,135000.000,0.005809,0.304401,0.029995,0.118142,0.118553,0.114913,...,0.3900,0.000289,2.8980,0.5160,0.708382,0.568101,0.06940,0.070019,0.497276,-inf


In [5]:
y = df["TARGET"].copy()
y.head(5)

0    1.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: TARGET, dtype: float64

In [19]:
z = df["TARGET"].values
z

array([1., 0., 0., ..., 0., 1., 0.])

In [20]:
w = df[['TARGET','AMT_CREDIT','AMT_GOODS_PRICE']]
w

,TARGET,AMT_CREDIT,AMT_GOODS_PRICE
0,1.0,406597.5,351000.0
1,0.0,1293502.5,1129500.0
2,0.0,135000.0,135000.0
3,0.0,312682.5,297000.0
4,0.0,513000.0,513000.0
5,0.0,490495.5,454500.0
6,0.0,1560726.0,1395000.0
7,0.0,1530000.0,1530000.0
8,0.0,1019610.0,913500.0
9,0.0,405000.0,405000.0


In [15]:
data3 = (data3 - data3.min())/(data3.max() - data3.min())

MemoryError: 